In [15]:
import shutil
import pathlib
import pandas as pd

In [14]:
their_md5_file = "idr0080-way-perturbation-20200805-s3.ms5sums"

their_df = pd.read_csv(their_md5_file, sep=" ").reset_index(drop=False).drop(["level_1"], axis="columns")
their_df.columns = ["md5", "file_name"]

their_df.file_name = (
    their_df.file_name.str.replace(
        "projects/2015_07_01_Cell_Health_Vazquez_Cancer_Broad/illumcorrected_CRISPR_PILOT_B1", "."
    )
)

their_df = their_df.assign(base_file=[x[-1] for x in their_df.file_name.str.split("/")])
their_df = their_df.loc[their_df.base_file.str.contains(".tiff"), :]
their_df = their_df.assign(plate=[x[2] for x in their_df.file_name.str.split("/")])
their_df = their_df.assign(plate_file=their_df.plate + "/" + their_df.base_file)

print(their_df.shape)
their_df.head()

(138235, 5)


,md5,file_name,base_file,plate,plate_file
2,d542db49afbc3494553c0e1e36704821,./images/SQ00014610/Images/r01c01f01p01-ch1sk1...,r01c01f01p01-ch1sk1fk1fl1.tiff,SQ00014610,SQ00014610/r01c01f01p01-ch1sk1fk1fl1.tiff
3,87ff2c74ca9f9ef6b3bc9fbf17598c28,./images/SQ00014610/Images/r01c01f01p01-ch2sk1...,r01c01f01p01-ch2sk1fk1fl1.tiff,SQ00014610,SQ00014610/r01c01f01p01-ch2sk1fk1fl1.tiff
4,3d5a58faeea266f0c967d02ae1df6a5a,./images/SQ00014610/Images/r01c01f01p01-ch3sk1...,r01c01f01p01-ch3sk1fk1fl1.tiff,SQ00014610,SQ00014610/r01c01f01p01-ch3sk1fk1fl1.tiff
5,5c972df70319832bf6b00239064e9676,./images/SQ00014610/Images/r01c01f01p01-ch4sk1...,r01c01f01p01-ch4sk1fk1fl1.tiff,SQ00014610,SQ00014610/r01c01f01p01-ch4sk1fk1fl1.tiff
6,7a9c245adec7cd69b054aff912951a86,./images/SQ00014610/Images/r01c01f01p01-ch5sk1...,r01c01f01p01-ch5sk1fk1fl1.tiff,SQ00014610,SQ00014610/r01c01f01p01-ch5sk1fk1fl1.tiff


In [22]:
append_data = []
try:
    shutil.copy2("test", ".")
except FileNotFoundError:
    append_data.append("test")
    
pd.DataFrame(append_data)

,0
0,test


In [3]:
all_images = "all_images.txt"

all_images_df = pd.read_csv(all_images, header=None)
all_images_df.columns = ["file_name"]

all_images_df = all_images_df.assign(base_file=[x[-1] for x in all_images_df.file_name.str.split("/")])
all_images_df = all_images_df.loc[all_images_df.base_file.str.contains(".tiff"), :]
all_images_df = all_images_df.assign(plate=[x[0].strip("./") for x in all_images_df.file_name.str.split("__")])
all_images_df = all_images_df.assign(plate_file=all_images_df.plate + "/" + all_images_df.base_file)

print(all_images_df.shape)
all_images_df.head()

(155520, 4)


,file_name,base_file,plate,plate_file
3,./SQ00014610__2016-06-16T00_38_35-Measurement2...,r01c01f01p01-ch1sk1fk1fl1.tiff,SQ00014610,SQ00014610/r01c01f01p01-ch1sk1fk1fl1.tiff
4,./SQ00014610__2016-06-16T00_38_35-Measurement2...,r01c01f01p01-ch2sk1fk1fl1.tiff,SQ00014610,SQ00014610/r01c01f01p01-ch2sk1fk1fl1.tiff
5,./SQ00014610__2016-06-16T00_38_35-Measurement2...,r01c01f01p01-ch3sk1fk1fl1.tiff,SQ00014610,SQ00014610/r01c01f01p01-ch3sk1fk1fl1.tiff
6,./SQ00014610__2016-06-16T00_38_35-Measurement2...,r01c01f01p01-ch4sk1fk1fl1.tiff,SQ00014610,SQ00014610/r01c01f01p01-ch4sk1fk1fl1.tiff
7,./SQ00014610__2016-06-16T00_38_35-Measurement2...,r01c01f01p01-ch5sk1fk1fl1.tiff,SQ00014610,SQ00014610/r01c01f01p01-ch5sk1fk1fl1.tiff


In [4]:
all_file_names = set(all_images_df.plate_file.tolist())
uploaded_file_names = set(their_df.plate_file.tolist())

missing_files = list(all_file_names.difference(uploaded_file_names))
print(len(missing_files))

17285


In [5]:
set([x.split("/")[1][6:9] for x in missing_files])

{'f01', 'f09'}

In [11]:
missing_images_df = all_images_df.loc[all_images_df.plate_file.isin(missing_files), :].reset_index(drop=True)

missing_images_df = missing_images_df.assign(f_channel=[x[6:9] for x in missing_images_df.base_file])

out_file = pathlib.Path("missing_images_failed_upload.tsv")
missing_images_df.to_csv(out_file, sep="\t", index=False)

print(missing_images_df.shape)
missing_images_df.head()

(17285, 5)


,file_name,base_file,plate,plate_file,f_channel
0,./SQ00014610__2016-06-16T00_38_35-Measurement2...,r01c01f09p01-ch1sk1fk1fl1.tiff,SQ00014610,SQ00014610/r01c01f09p01-ch1sk1fk1fl1.tiff,f09
1,./SQ00014610__2016-06-16T00_38_35-Measurement2...,r01c01f09p01-ch2sk1fk1fl1.tiff,SQ00014610,SQ00014610/r01c01f09p01-ch2sk1fk1fl1.tiff,f09
2,./SQ00014610__2016-06-16T00_38_35-Measurement2...,r01c01f09p01-ch3sk1fk1fl1.tiff,SQ00014610,SQ00014610/r01c01f09p01-ch3sk1fk1fl1.tiff,f09
3,./SQ00014610__2016-06-16T00_38_35-Measurement2...,r01c01f09p01-ch4sk1fk1fl1.tiff,SQ00014610,SQ00014610/r01c01f09p01-ch4sk1fk1fl1.tiff,f09
4,./SQ00014610__2016-06-16T00_38_35-Measurement2...,r01c01f09p01-ch5sk1fk1fl1.tiff,SQ00014610,SQ00014610/r01c01f09p01-ch5sk1fk1fl1.tiff,f09


In [12]:
# Confirmed that the missing files are 
missing_images_df.query("f_channel == 'f01'")

,file_name,base_file,plate,plate_file,f_channel
1915,./SQ00014610__2016-06-16T00_38_35-Measurement2...,r16c24f01p01-ch1sk1fk1fl1.tiff,SQ00014610,SQ00014610/r16c24f01p01-ch1sk1fk1fl1.tiff,f01
1916,./SQ00014610__2016-06-16T00_38_35-Measurement2...,r16c24f01p01-ch2sk1fk1fl1.tiff,SQ00014610,SQ00014610/r16c24f01p01-ch2sk1fk1fl1.tiff,f01
1917,./SQ00014610__2016-06-16T00_38_35-Measurement2...,r16c24f01p01-ch3sk1fk1fl1.tiff,SQ00014610,SQ00014610/r16c24f01p01-ch3sk1fk1fl1.tiff,f01
1918,./SQ00014610__2016-06-16T00_38_35-Measurement2...,r16c24f01p01-ch4sk1fk1fl1.tiff,SQ00014610,SQ00014610/r16c24f01p01-ch4sk1fk1fl1.tiff,f01
1919,./SQ00014610__2016-06-16T00_38_35-Measurement2...,r16c24f01p01-ch5sk1fk1fl1.tiff,SQ00014610,SQ00014610/r16c24f01p01-ch5sk1fk1fl1.tiff,f01
